In [76]:
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import itertools
import glob
import pickle
import matplotlib.pyplot as plt


In [77]:
def getTrainData(path_files):
    #Import Mineral Balanced Data!!!!
    
    input_files = glob.glob(path_files + "*.csv")

    min_in = []

    for filename in input_files:
        df = pd.read_csv(filename, index_col=None, header=0, encoding = "ISO-8859-1")
        min_in.append(df)

    df_all = pd.concat(min_in, axis=0, ignore_index=True)
    df = df_all.drop(columns = ['Unnamed: 0', 'X1', 'id', 'SAMPLE', 'GROUP', 'MINERAL', 'ROCK','X1_1'])
    df = df.drop(columns = ["H20"])

    return df, df_all

    
def test_acc(X,y, model,plot=True,modelName='rf'):
    # Function to test the accuracy
    n_nodes = []
    max_depths = []

    for ind_tree in model.estimators_:
        n_nodes.append(ind_tree.tree_.node_count)
        max_depths.append(ind_tree.tree_.max_depth)

    print(f'Average number of nodes {int(np.mean(n_nodes))}')
    print(f'Average maximum depth {int(np.mean(max_depths))}')
    
    feature_list = list(X.columns)
    # Get numerical feature importances
    importances = list(model.feature_importances_)
    # List of tuples with variable and importance
    feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(feature_list, importances)]
    # Sort the feature importances by most important first
    feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)
    # Print out the feature and importances 
    [print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances];

    y_pred = model.predict(X)
    
    cm = confusion_matrix(y, y_pred)
    if plot:
        if modelName == 'rf':
            qmin.plot_confusion_matrix(cm, classes = np.unique(y),
                                  title = 'Confusion Matrix',normalize=True)
        else:
            qmin.plot_confusion_matrix(cm, classes = np.unique(y),
                                  title = 'Confusion Matrix',normalize=True,
                                 output='../figures/RF_'+modelName+'_confusion_matrix.png')
        print(classification_report(y,y_pred))
    print(accuracy_score(y, y_pred))
    return accuracy_score(y, y_pred)

def save_Model(model, model_name):
    path = '../model_py/'
    pickle.dump(model, open(path+model_name, 'wb'))
    print("Model Saved ...\n"+path+model_name)
    
def randomForestBuilder(trainData,labels, acc_test=True, saveModel=True, nameModel='RF'):
    # Separate 30% for test, training in 70% of Data!
    train, test, train_labels, test_labels = train_test_split(trainData, labels, 
                                                              stratify = labels,
                                                              test_size = 0.3)
    model = RandomForestClassifier(n_estimators=50, 
                               max_features = 'sqrt',
                               n_jobs=-1, verbose = 1,
                               oob_score=True)

    # Fit on training data
    model.fit(train, train_labels)
    
    if acc_test:
        acc = test_acc(test,test_labels,model,modelName=nameModel)
    
    if saveModel:
        save_Model(model, nameModel+'.pkl')
    
    return model, test_acc(test,test_labels,model,plot=False)

def _cleanDataFrame(df):
    df = df.drop(columns = ['Unnamed: 0', 'X1', 'id', 'SAMPLE', 'GROUP', 'ROCK','X1_1'])
    df = df.drop(columns = ["H20"])
    
    return df

def createMineralModel(data,namegroup):
    #get Dataframe to create multiple mineral models
    
    labels = labels = np.array(data.pop('MINERAL'))
    trainData = _cleanDataFrame(data)    
    mineralModel, acc = randomForestBuilder(trainData,labels,acc_test=False,
                                     saveModel=False, nameModel=namegroup)
    return mineralModel,acc
    
    

def modelCreate():
    models = {}
    
    trainData, allData = getTrainData('../data_train/')
    trainFeatures = trainData.columns
    labels = np.array(allData.pop('GROUP'))
    allData['GROUP'] = labels

    #Group Model Classify
    groupModel, acc = randomForestBuilder(trainData,labels,acc_test=False,
                                     saveModel=True, nameModel='RF_GROUP')
    models['GROUP'] = groupModel
    
    groups = allData['GROUP'].unique() # Get all Gruops for mineral classification
    
    # Mineral Model Classify
    accs = []
    for group in groups:
        print('Training .... '+group)
        mineralData = allData[allData['GROUP']== group]
        models[group], acc = createMineralModel(mineralData,group)
        accs.append([group,acc])
    
    print(accs)
    return models,trainFeatures


models, trainFeatures = modelCreate()
models['Train Features'] = trainFeatures.values.tolist()
#models['Train Features'] = ['SIO2','TIO2','AL2O3','CR2O3','FEOT','CAO','MGO','MNO','K2O',\
# 'NA2O','P2O5','F','CL','NIO','CUO','COO','ZNO','PBO','S','ZRO2', 'AS']
path = '../model_py/'
model_name = 'allmodels'

with open(path+model_name+'.pkl', 'wb') as f:
    pickle.dump(models, f)
# pickle.dump(models, open(path+model_name+'.pkl', 'wb'))
print("Model Saved ...\n"+path+model_name)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:    0.2s finished


Model Saved ...
../model_py/RF_GROUP.pkl
Average number of nodes 339
Average maximum depth 15
Variable: SIO2                 Importance: 0.15
Variable: AL2O3                Importance: 0.1
Variable: S                    Importance: 0.1
Variable: F                    Importance: 0.07
Variable: ZRO2                 Importance: 0.07
Variable: CAO                  Importance: 0.06
Variable: K2O                  Importance: 0.06
Variable: MGO                  Importance: 0.05
Variable: NA2O                 Importance: 0.05
Variable: CL                   Importance: 0.05
Variable: FEOT                 Importance: 0.04
Variable: COO                  Importance: 0.04
Variable: ZNO                  Importance: 0.04
Variable: TIO2                 Importance: 0.03
Variable: MNO                  Importance: 0.02
Variable: P2O5                 Importance: 0.02
Variable: NIO                  Importance: 0.02
Variable: CUO                  Importance: 0.02
Variable: CR2O3                Importance: 0

[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done  50 out of  50 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.


0.9923167848699763
Training .... AMPHIBOLES
Average number of nodes 309
Average maximum depth 15


[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:    0.1s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done  50 out of  50 | elapsed:    0.0s finished


...

Average number of nodes 1
Average maximum depth 0
Variable: SIO2                 Importance: 0.0
Variable: TIO2                 Importance: 0.0
Variable: AL2O3                Importance: 0.0
Variable: CR2O3                Importance: 0.0
Variable: FEOT                 Importance: 0.0
Variable: CAO                  Importance: 0.0
Variable: MGO                  Importance: 0.0
Variable: MNO                  Importance: 0.0
Variable: K2O                  Importance: 0.0
Variable: NA2O                 Importance: 0.0
Variable: P2O5                 Importance: 0.0
Variable: F                    Importance: 0.0
Variable: CL                   Importance: 0.0
Variable: NIO                  Importance: 0.0
Variable: CUO                  Importance: 0.0
Variable: COO                  Importance: 0.0
Variable: ZNO                  Importance: 0.0
Variable: PBO                  Importance: 0.0
Variable: S                    Importance: 0.0
Variable: ZRO2                 Importance: 0.0
Variable: 

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:    0.0s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done  50 out of  50 | elapsed:    0.0s finished


Average number of nodes 1
Average maximum depth 0
Variable: SIO2                 Importance: 0.0
Variable: TIO2                 Importance: 0.0
Variable: AL2O3                Importance: 0.0
Variable: CR2O3                Importance: 0.0
Variable: FEOT                 Importance: 0.0
Variable: CAO                  Importance: 0.0
Variable: MGO                  Importance: 0.0
Variable: MNO                  Importance: 0.0
Variable: K2O                  Importance: 0.0
Variable: NA2O                 Importance: 0.0
Variable: P2O5                 Importance: 0.0
Variable: F                    Importance: 0.0
Variable: CL                   Importance: 0.0
Variable: NIO                  Importance: 0.0
Variable: CUO                  Importance: 0.0
Variable: COO                  Importance: 0.0
Variable: ZNO                  Importance: 0.0
Variable: PBO                  Importance: 0.0
Variable: S                    Importance: 0.0
Variable: ZRO2                 Importance: 0.0
Variable: 

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:    0.0s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done  50 out of  50 | elapsed:    0.0s finished


Average number of nodes 1
Average maximum depth 0
Variable: SIO2                 Importance: 0.0
Variable: TIO2                 Importance: 0.0
Variable: AL2O3                Importance: 0.0
Variable: CR2O3                Importance: 0.0
Variable: FEOT                 Importance: 0.0
Variable: CAO                  Importance: 0.0
Variable: MGO                  Importance: 0.0
Variable: MNO                  Importance: 0.0
Variable: K2O                  Importance: 0.0
Variable: NA2O                 Importance: 0.0
Variable: P2O5                 Importance: 0.0
Variable: F                    Importance: 0.0
Variable: CL                   Importance: 0.0
Variable: NIO                  Importance: 0.0
Variable: CUO                  Importance: 0.0
Variable: COO                  Importance: 0.0
Variable: ZNO                  Importance: 0.0
Variable: PBO                  Importance: 0.0
Variable: S                    Importance: 0.0
Variable: ZRO2                 Importance: 0.0
Variable: 

In [78]:
accs = [['AMPHIBOLES', 0.8045977011494253], ['CARBONATE', 0.8888888888888888], ['CLAY', 0.8271604938271605], ['FELDSPAR', 0.8888888888888888], ['FELDSPATHOID', 0.8787878787878788], ['GARNET', 0.8611111111111112], ['MICA', 0.9281045751633987], ['OLIVINE', 0.9333333333333333], ['PYROXENE', 0.85], ['SPINEL', 0.7373737373737373], ['SULFIDE', 0.9354838709677419], ['APATITE', 1.0], ['ILMENITE', 1.0], ['PEROVSKITE', 1.0], ['QUARTZ', 1.0], ['TITANITE', 1.0], ['ZIRCON', 1.0]]
for i in accs:
    print('%s, %.3f'%(i[0],i[1]))

AMPHIBOLES, 0.805
CARBONATE, 0.889
CLAY, 0.827
FELDSPAR, 0.889
FELDSPATHOID, 0.879
GARNET, 0.861
MICA, 0.928
OLIVINE, 0.933
PYROXENE, 0.850
SPINEL, 0.737
SULFIDE, 0.935
APATITE, 1.000
ILMENITE, 1.000
PEROVSKITE, 1.000
QUARTZ, 1.000
TITANITE, 1.000
ZIRCON, 1.000
